In [48]:
# Importing libraries
import pandas as pd
import requests, csv

In [50]:
# Asking for the wanted month
month = str(input('Write the number of the month as for example "05" or "09". Then press Enter.'))

Write the number of the month as for example "05" or "09". Then press Enter. 05


In [51]:
# Asking for the wanted year
year = str(input('Write the year as for example "2019". Then press Enter.'))

Write the year as for example "2019". Then press Enter. 2019


In [52]:
# Creating a URL to the wanted data
url =  "http://dev.hsl.fi/citybikes/od-trips-"+ year+ "/"+year +"-"+month+".csv"

In [53]:
# Reading the data from the URL to a dataframe
data = pd.read_csv(url)

In [55]:
print(str(len(data)) + " trips were made in " + month + "/" + year+ ".")

599730 trips were made in 05/2019.


In [59]:
# Setting depature and return time to datetime format
data['departure_time'] = pd.to_datetime(data['Departure'], format='%Y-%m-%dT%H:%M:%S')
data['return_time'] = pd.to_datetime(data['Return'], format='%Y-%m-%dT%H:%M:%S')

In [70]:
# New dataframe with depature time as datetimeindex
dep_index_data = data
dep_index_data['departure_time_2'] = dep_index_data['departure_time']
dep_index_data.set_index(['departure_time'], inplace=True)

599730

In [71]:
# Selecting rows with departure time between 00:00 and 05:00
departures = dep_index_data.between_time('00:00','05:00')

22022

In [73]:
# New dataframe with return time as datetimeindex
ret_index_data = departures.set_index(['return_time'])

In [74]:
# Selecting rows with return time between 00:00 and 05:00
returns = ret_index_data.between_time('00:00','05:00')

In [76]:
# New dataframe with only trips between 00:00 and 05:00
night = returns.reset_index()

In [77]:
# Dropping unnecessary columns
night.drop(columns=['Departure','Return'], inplace=True)

In [79]:
night.head(5)

,return_time,Departure station id,Departure station name,Return station id,Return station name,Covered distance (m),Duration (sec.),departure_time_2
0,2019-05-31 04:47:30,129,Pernajantie,129.0,Pernajantie,2.0,23,2019-05-31 04:47:03
1,2019-05-31 04:50:29,111,Esterinportti,82.0,Töölöntulli,1206.0,270,2019-05-31 04:45:58
2,2019-05-31 04:55:07,104,Kriikunakuja,213.0,Huopalahden asema,1990.0,560,2019-05-31 04:45:42
3,2019-05-31 04:50:26,62,Messitytönkatu,21.0,Töölönlahdenkatu,2025.0,590,2019-05-31 04:40:31
4,2019-05-31 04:39:45,100,Teljäntie,101.0,Munkkivuoren ostoskeskus,804.0,264,2019-05-31 04:35:26
